# <font color='blue'> Random forest classification model </font>

## Read input data, including ground truth data and images

In [ ]:
# Import GDAL, NumPy, and matplotlib
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# Read in the image and ground data recalled as roi
img_ds = gdal.Open(r"path", gdal.GA_ReadOnly)
roi_ds = gdal.Open(r"path", gdal.GA_ReadOnly)

# gdal_array to read raster data as numeric array from file
img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
              gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType)) 

for b in range (img.shape[2]):
    img[:, :, b] = img_ds.GetRasterBand(b+1).ReadAsArray()

roi = roi_ds.GetRasterBand(1).ReadAsArray().astype(np.int16)


# Display one band of image and ground truth (roi) data
plt.subplot(121)
plt.imshow(img[:, :, -2], cmap=plt.cm.Greys_r, vmin=-2000, vmax=10000) # vmin and vmax is defined based on your data
plt.title('Input Data')

plt.subplot(122)
plt.imshow(roi, cmap=plt.cm.Spectral)
plt.title("Ground Truth Data")

plt.show()